# Working with Qiskit

To install qiskit, just run `!pip install qiskit` (note: if you have an older version of qiskit, run `!pip install qiskit --upgrade` instead.


In [1]:
!pip install qiskit


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install --upgrade pip


SyntaxError: invalid syntax (3919076351.py, line 1)

In [4]:
from qiskit import *

In [9]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Clifford #This is the Clifford Object, which internally stores the operator as a 2n x 2n+1 tableau.
from qiskit.circuit.library import *

# The Clifford Object

The idea essentially is that for a quantum circuit object `qc = QuantumCircuit(n)`, the object `cliff = Clifford(qc)` stores the Clifford tableau corresponding to the Clifford Circuit. An example is shown below:

In [6]:
qc = QuantumCircuit(2)
qc.h(0)
qc.z(0)
qc.h(0)

qc.draw()

┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ Z ├┤ H ├
     └───┘└───┘└───┘
q_1: ───────────────

In [10]:
cliff = Clifford(qc)
for i in cliff.tableau:
    print(i)

[ True False False False False]
[False  True False False False]
[False False  True False  True]
[False False False  True False]


So, each Clifford operator can be identified uniquely by its' tableau. For example:

In [12]:
qc_ = QuantumCircuit(2)
qc_.x(0) #Note: HZH = X

qc_.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────

In [13]:
cliff_ = Clifford(qc_)
for i in cliff_.tableau:
    print(i)

print('Are both operators equal? Ans:', cliff==cliff_)

[ True False False False False]
[False  True False False False]
[False False  True False  True]
[False False False  True False]
Are both operators equal? Ans: True


Hence, we can identify each VERTEX with a tableau. The quantum circuits are essentially words.

# Approach 

For a given quantum circuit `qc = QuantumCircuit(n)` with clifford `cliff = Clifford(qc)`, we need to find a minimised circuit `opqc` such that `Clifford(opqc) = cliff`. 

We use a `QuantumCircuit` object for word storage and the `Clifford` object to identify the vertices. 

## Identity element:

We start with an IDENTITY CIRCUIT `qc = QuantumCircuit(n)` (without any gates), and it's tableau representation is `c_ = Clifford(qc)`. This element is constantly updated. 

## Traversal:

We can do this in two ways:

1. Keep updating `qc` with instructions (`qc.h(i)`,`qc.s(i)`,`qc.cx(i,j)`), and at each step check `Clifford(qc)` to identify the vertex. This might be easy to implement, but this would mean generating a NEW TABLEAU for each version of the circuit. This can be unnecessarily time-consuming.
2. Use `qc` solely as a word-storage circuit, and manually update `c_` via Clifford operation compositions. This might look like changing both manually, but it makes more sense.

* Note: in both of these, you WILL have to keep a track of the elements already traversed... *  

We follow approach #2 here. 
We go on to define some functions so as to make our job easier.

We define the following functions, which correspond to the generators $H_{i} , S_{i}, CX_{i,j}$:

1. `addH(qc,c_,i)` : Adds H gate to circuit at position i and returns an updated tableau, corresponding to generator $H_i$
2. `addS(qc,c_,i)` : Adds S gate to circuit at position i and returns an updated tableau, corresponding to generator $S_i$
3. `addCX(qc,c_,i,j)` : Adds CX gate to circuit with control i and target j and returns an updated tableau, corresponding to generator $C_{i,j}$

In case one wants to traverse backwards i.e. remove circuit elements, that can also be done. One can delete a circuit element by performing the operation `del(qc.data[-1])` , but you will have to parallely update c_ using a separate array used to track the elements already traversed.

(Alternatively one can also implement the deletion function but then it MIGHT get a little too unnecessarily complicated.)

In [14]:
def addH(qc,cliff,pos): 
    qc.h(pos) #add H to qc
    num = cliff.num_qubits
    st = 'I'*(num-pos-1)+'H'+'I'*pos
    cjoin = cliff.compose(Clifford.from_label(st)) #Clifford Operator Composition
    return cjoin

def addS(qc,cliff,pos):
    qc.s(pos) #add S to qc
    num = cliff.num_qubits
    st = 'I'*(num-pos-1)+'S'+'I'*pos
    cjoin = cliff.compose(Clifford.from_label(st)) #Clifford Operator Composition    
    return cjoin

def addCX(qc,cliff,ctrl,tgt):
    qc.cx(ctrl,tgt) #add CZ to qc
    num = cliff.num_qubits
    cxc = QuantumCircuit(num)  #We create the CX circuit since the label call doesnt exist for multi-qubit... 
    cxc.cx(ctrl,tgt)
    cjoin = cliff.compose(Clifford(cxc)) #Clifford Operation Composition
    return cjoin

# The circuit is modified inside the function, and an updated Clifford operation is returned (not affecting the original one, since you might wanna decide how to implement this when working with the graph). 

In [15]:
qc = QuantumCircuit(3)
c_ = Clifford(qc)

traversed = [] #Or whatever structure you use to store / keep track of the array. 

print(qc,c_.tableau)


traversed.append(c_)

     
q_0: 
     
q_1: 
     
q_2: 
      [[ True False False False False False False]
 [False  True False False False False False]
 [False False  True False False False False]
 [False False False  True False False False]
 [False False False False  True False False]
 [False False False False False  True False]]


In [16]:
#Add a hadamard

c_ = addH(qc,c_,1) 
traversed.append(c_)

print(qc,c_.tableau) 

          
q_0: ─────
     ┌───┐
q_1: ┤ H ├
     └───┘
q_2: ─────
           [[ True False False False False False False]
 [False False False False  True False False]
 [False False  True False False False False]
 [False False False  True False False False]
 [False  True False False False False False]
 [False False False False False  True False]]


# A simple implementation

The following code was mostly generated by ChatGPT but what it does is an implementation of a BFS type algorithm, I would suggest we do a comparitive study of all algorithms available to us. 



In [17]:
qc_m=QuantumCircuit(2)
qc_m.h([0,1])
qc_m.cx(1,0)
qc_m.h([0,1])
qc_m.s(0)
qc_m.s(0)
qc_m.s(0)
qc_m.s(0)
qc_m.s(0)
qc_m.s(1)
qc_m.s(1)

qc_m.draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ X ├┤ H ├┤ S ├┤ S ├┤ S ├┤ S ├┤ S ├
     ├───┤└─┬─┘├───┤├───┤├───┤└───┘└───┘└───┘
q_1: ┤ H ├──■──┤ H ├┤ S ├┤ S ├───────────────
     └───┘     └───┘└───┘└───┘

In [18]:
from collections import deque

def cliff_hash(cliff):
    """Return a hashable representation of a Clifford."""
    return tuple(map(tuple, cliff.tableau))

def minimize(qc_target):
    n = qc_target.num_qubits
    target = Clifford(qc_target)
    id_cliff = Clifford(QuantumCircuit(n))
    
    queue = deque([(QuantumCircuit(n), id_cliff, [])])
    visited = {cliff_hash(id_cliff)}
    
    target_hash = cliff_hash(target)
    
    while queue:
        qc, cliff, path = queue.popleft()
        
        if cliff_hash(cliff) == target_hash:
            return qc, path
        
        # Expand
        for q in range(n):
            new_qc = qc.copy(); new_cliff = addH(new_qc, cliff, q)
            h = cliff_hash(new_cliff)
            if h not in visited:
                visited.add(h)
                queue.append((new_qc, new_cliff, path+[("H", q)]))
            
            new_qc = qc.copy(); new_cliff = addS(new_qc, cliff, q)
            h = cliff_hash(new_cliff)
            if h not in visited:
                visited.add(h)
                queue.append((new_qc, new_cliff, path+[("S", q)]))
        
        for c in range(n):
            for t in range(n):
                if c != t:
                    new_qc = qc.copy(); new_cliff = addCX(new_qc, cliff, c, t)
                    h = cliff_hash(new_cliff)
                    if h not in visited:
                        visited.add(h)
                        queue.append((new_qc, new_cliff, path+[("CX", (c, t))]))

qnew = minimize(qc_m)

In [19]:
qnew[0].draw()

┌───┐               
q_0: ┤ S ├──■────────────
     └───┘┌─┴─┐┌───┐┌───┐
q_1: ─────┤ X ├┤ S ├┤ S ├
          └───┘└───┘└───┘

Below is an implementation of an on-the-fly Dijkstra's algorithm. 

In [20]:
import heapq
from qiskit import QuantumCircuit
from qiskit.quantum_info import Clifford

def clifford_key(cliff):
    """Return a compact, hashable representation of a Clifford."""
    return cliff.tableau.tobytes()

def dijkstra_minimize(qc_target, weights=None):
    n = qc_target.num_qubits
    target_cliff = Clifford(qc_target)
    target_key = clifford_key(target_cliff)

    # Default weights
    if weights is None:
        weights = {"H": 1, "S": 1, "CX": 2}

    # Priority queue: (cost, counter, circuit, clifford, path)
    pq = []
    counter = 0  # unique index to avoid comparison issues

    start = QuantumCircuit(n)
    start_cliff = Clifford(start)
    heapq.heappush(pq, (0, counter, start, start_cliff, []))

    dist = {clifford_key(start_cliff): 0}
    visited = set()

    while pq:
        cost, _, qc, cliff, path = heapq.heappop(pq)
        cliff_key = clifford_key(cliff)

        if cliff_key == target_key:
            return qc, path, cost

        if cliff_key in visited:
            continue
        visited.add(cliff_key)

        # Generate neighbors
        for q in range(n):
            # H
            new_qc = qc.copy()
            new_qc.h(q)
            new_cliff = Clifford(new_qc)
            new_cost = cost + weights["H"]
            new_key = clifford_key(new_cliff)
            if new_cost < dist.get(new_key, float("inf")):
                dist[new_key] = new_cost
                counter += 1
                heapq.heappush(pq, (new_cost, counter, new_qc, new_cliff, path+[("H", q)]))

            # S
            new_qc = qc.copy()
            new_qc.s(q)
            new_cliff = Clifford(new_qc)
            new_cost = cost + weights["S"]
            new_key = clifford_key(new_cliff)
            if new_cost < dist.get(new_key, float("inf")):
                dist[new_key] = new_cost
                counter += 1
                heapq.heappush(pq, (new_cost, counter, new_qc, new_cliff, path+[("S", q)]))

        for c in range(n):
            for t in range(n):
                if c == t: 
                    continue
                new_qc = qc.copy()
                new_qc.cx(c, t)
                new_cliff = Clifford(new_qc)
                new_cost = cost + weights["CX"]
                new_key = clifford_key(new_cliff)
                if new_cost < dist.get(new_key, float("inf")):
                    dist[new_key] = new_cost
                    counter += 1
                    heapq.heappush(pq, (new_cost, counter, new_qc, new_cliff, path+[("CX", c, t)]))

In [21]:
qc_new = dijkstra_minimize(qc_m)
qc_new[0].draw()

┌───┐               
q_0: ┤ S ├──■────────────
     └───┘┌─┴─┐┌───┐┌───┐
q_1: ─────┤ X ├┤ S ├┤ S ├
          └───┘└───┘└───┘

In [22]:
q1 = QuantumCircuit(1)
q1.z(0)
q1.x(0)
q2 = QuantumCircuit(1)
q2.y(0)
print(Clifford(q1).tableau)
print(Clifford(q2).tableau)


Clifford(q1).tableau.tobytes() ==  Clifford(q2).tableau.tobytes() , q1==q2

[[ True False  True]
 [False  True  True]]
[[ True False  True]
 [False  True  True]]


(True, False)

# Cayley Graph Generation


In [23]:


H = lambda pos,num: Clifford.from_label('I'*(num-pos-1)+'H'+'I'*pos)
S = lambda pos,num: Clifford.from_label('I'*(num-pos-1)+'S'+'I'*pos)

def CX(ctrl,tgt,num):
    if ctrl == tgt: return None
    else:
        cxc = QuantumCircuit(num)  #We create the CX circuit since the label call doesnt exist for multi-qubit... 
        cxc.cx(ctrl,tgt)
        return (Clifford(cxc)) #Clifford Operation Composition

#Based on number of circuits, we have the following generators:

n = 2

H_n = [ ('H'+str(i),H(i,n)) for i in range(n)]
S_n = [('S'+str(i),S(i,n)) for i in range(n)]

CX_ijn = []
for i in range(n):
    for j in range(n):
        if i!=j: CX_ijn.append(('CX'+str(i)+str(j),CX(i,j,n)))

generators = H_n+CX_ijn




In [24]:
generators

[('H0',
  Clifford(array([[False, False,  True, False, False],
         [False,  True, False, False, False],
         [ True, False, False, False, False],
         [False, False, False,  True, False]]))),
 ('H1',
  Clifford(array([[ True, False, False, False, False],
         [False, False, False,  True, False],
         [False, False,  True, False, False],
         [False,  True, False, False, False]]))),
 ('CX01',
  Clifford(array([[ True,  True, False, False, False],
         [False,  True, False, False, False],
         [False, False,  True, False, False],
         [False, False,  True,  True, False]]))),
 ('CX10',
  Clifford(array([[ True, False, False, False, False],
         [ True,  True, False, False, False],
         [False, False,  True,  True, False],
         [False, False, False,  True, False]])))]

In [25]:
def clifford_key(cliff):
    return cliff.tableau.tobytes()


id_cliff = Clifford.from_circuit(QuantumCircuit(2))
start_key = clifford_key(id_cliff)  # compact, hashable
identity=id_cliff

In [26]:
class Node:
    def __init__(self, clifford, name=""):
        self.clifford = clifford
        self.key = clifford_key(clifford)
        self.edges = {}  # generator_name -> neighbor Node
        self.name = name  # optional: keep track of generator used

def build_cayley_graph(generators, start, max_nodes=None):
    start_node = Node(start, "identity")
    nodes = {start_node.key: start_node}
    queue = deque([start_node])

    while queue:
        current = queue.popleft()

        for gen_name, gen_op in generators:
            new_cliff = current.clifford.compose(gen_op)
            new_key = clifford_key(new_cliff)

            if new_key not in nodes:
                new_node = Node(new_cliff, gen_name)
                nodes[new_key] = new_node
                queue.append(new_node)

            # connect edge
            current.edges[gen_name] = (nodes[new_key], 1)

        if max_nodes and len(nodes) >= max_nodes:
            break

    return nodes


# --- Example usage ---
graph = build_cayley_graph(generators, identity)

print("Generated nodes:", len(graph))
for k, node in list(graph.items())[:5]:
    print("Node:", node.name, " | Outgoing edges:", list(node.edges.keys()))

Generated nodes: 1152
Node: identity  | Outgoing edges: ['H0', 'H1', 'CX01', 'CX10']
Node: H0  | Outgoing edges: ['H0', 'H1', 'CX01', 'CX10']
Node: H1  | Outgoing edges: ['H0', 'H1', 'CX01', 'CX10']
Node: CX01  | Outgoing edges: ['H0', 'H1', 'CX01', 'CX10']
Node: CX10  | Outgoing edges: ['H0', 'H1', 'CX01', 'CX10']


In [27]:
len(graph)

1152

In [28]:
def dijkstra(nodes, start_key, target_key):
    dist = {k: float("inf") for k in nodes}
    prev = {k: None for k in nodes}
    dist[start_key] = 0

    heap = [(0, start_key)]
    while heap:
        d, u = heapq.heappop(heap)
        if d > dist[u]:
            continue
        if u == target_key:
            break
        for gen_name, (neighbor, w) in nodes[u].edges.items():
            v = neighbor.key
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                prev[v] = (u, gen_name)
                heapq.heappush(heap, (dist[v], v))

    # reconstruct path
    if dist[target_key] == float("inf"):
        return None  # no path
    path = []
    u = target_key
    while prev[u]:
        u_prev, gen = prev[u]
        path.append(gen)
        u = u_prev
    return list(reversed(path)), dist[target_key]

In [32]:
graph = build_cayley_graph(generators, identity)
example_qft = QuantumCircuit(2)

example_qft.h(0)
example_qft.h(0)
example_qft.cx(0, 1)
example_qft.h(1)
example_qft.h(1)

print("Naive QFT circuit:")
print(example_qft)

# --- Convert to Clifford ---
target = Clifford(example_qft)

result = dijkstra(graph, start_key=clifford_key(identity), target_key=clifford_key(target))

if result is None:
    print("No path found.")
else:
    path, dist = result
    print("Naive construction: H0 -> H0")
    print("Shortest circuit uses", dist, "gates:")
    print(" -> ".join(path) if path else "[identity]")

    qc = QuantumCircuit(n)
    for gate in path:
        if gate.startswith("H"):
            pos = int(gate[1])
            qc.h(pos)
        elif gate.startswith("S"):
            pos = int(gate[1])
            qc.s(pos)
        elif gate.startswith("CX"):
            ctrl, tgt = int(gate[2]), int(gate[3])
            qc.cx(ctrl, tgt)

    print("\nEquivalent reduced circuit:\n")
    print(qc)


Naive QFT circuit:
     ┌───┐┌───┐               
q_0: ┤ H ├┤ H ├──■────────────
     └───┘└───┘┌─┴─┐┌───┐┌───┐
q_1: ──────────┤ X ├┤ H ├┤ H ├
               └───┘└───┘└───┘
Naive construction: H0 -> H0
Shortest circuit uses 1 gates:
CX01

Equivalent reduced circuit:

          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
